# 深度卷积模型：案例分析

## 1. 案例分析

### 1.1 为什么要进行案例分析

过去几年来，计算机视觉领域的主要工作，就在研究如何拼装卷积层、池化层、全连接层等等卷积神经网络模型的基本组件，组合出较为有效的模型，有效的模型通常可以在不同的计算机视觉模型之间通用。因此，通过学习这些模型实例，可以对深度卷积模型有更好的理解。

### 1.2 经典网络

LeNet-5，用来处理灰度图片的数字识别问题。
    - 在LeNet-5的年代，补全的技巧还不常用，所有卷积层都不补全
    - 池化层在当时更多采用的是平均池化，目前最大池化更为常见
    - 模型大约有60000个参数，相对于现代神经网络，规模较小
    - 随着模型的深入，高度和宽度不断减小，通道数不断增加，这个实践沿用至今
    - 当时的激活函数采用的是sigmoid或者tanh，还没有使用ReLU
    - 当时单个过滤器，并不会处理所有的通道，这点和现在也不一样
    - 当时在池化层后面会接一个非线性的激活层，和现在不同

![LeNet-5.png](img/LeNet-5.png)

AlexNet，真正使得计算机视觉领域开始重视神经网络
    - 已经开始使用ReLU作为激活函数
    - 大约6000万的参数，中等规模
    - 当时的GPU计算能力还不够强，论文里涉及两个GPU的通信
    - 用到了Local Response Normalization(LRN)的技巧，这个技巧目前已经不常用了
    - same padding，同一补全

![AlexNet.png](img/AlexNet.png)

VGG-16，模式更为明确
    - 固定3×3的过滤器，步长s=1，同一补全；最大池化层2×2，步长s=2
    - 每次卷积层（连续两层）后，通道数翻倍
    - 每次池化层后，高度和宽度减半
    - 大约有1.38亿的参数，现代规模的神经网络
    - VGG-16中的16是值整个神经网路中共有16层带有参数的层，另外还有一个更大的VGG-19版本，不常用

![VGG-16.png](img/VGG-16.png)

### 1.3 残差网络 ResNets

由于梯度消失和梯度爆炸问题，训练非常大的神经网络通常非常困难。而残差网络解决了这个问题。残差网络的基本组成结构如下：
![Residual block.png](img/Residual block.png)

由于计算能力的问题，普通神经网络（Plain Network）在实践中随着层数增多，并不能获得更好的效果。而残差神经网络使训练层数非常多的模型成为可能。
![Residual Network.png](img/Residual Network.png)

### 1.4 残差网络为什么有效

假设现在有一个 $l$ 层的普通神经网络，输出结果 $a^{[l]}$。我们在其之后，又增加了两层残差Block，得到输出 $a^{[l+2]}$，根据上面的公式，$a^{[l+2]}=g(z^{[l+2]}+a^{[l]})=g(W^{[l+2]}a^{[l+1]}+b^{[l+2]}+a^{[l]})$。

假设我们对整个网络进行了L2正则化，那么$W^{[l+2]}$ 和 $b^{[l+2]}$ 就会约等于0，使得 $a^{[l+2]}$ 约等于 $a^{[l]}$。也即是说，在最坏的情况下，增加了两层残差Block，也只会使得神经网络的输出相同。而较好的情况下，我们可以训练出更好的模型。

值得注意的是，$z^{[l+2]}$ 和 $a^{[l]}$ 要可加，这两个矩阵的维度需要相同。因此，残差网络通常配合着同一补全的卷积层来使用。而对于池化层，则会需要增加一个 $W_s$ 的矩阵，和 $a^{[l]}$ 相乘后再进行加运算。

### 1.5 网络中的网络，1×1卷积

1×1的卷积在单一通道下看似乎没什么用，但是当通道数多了之后，1×1的卷积实际上可以对跨通道之间的数据项进行非线性组合。
![Why does a 1×1 convolution do.png](img/Why does a 1×1 convolution do.png)

1×1的卷积还可以对通道数进行缩减（就像池化层对高度和宽度进行缩减），当然如果愿意的化，1×1卷积也可以增加通道数。
![Using 1×1 convolutions.png](img/Using 1×1 convolutions.png)

### 1.6 Inception网络的设想

避免考虑要用1×1的卷积，还是3×3的卷积，还是池化层。直接将所有这些，叠加到同一层网络中，让模型自己去学习参数。

![Motivation for inception network.png](img/Motivation for inception network.png)

5×5的卷积，可能会引入比较大的计算量。这时，使用1×1的卷积，在中间做一层瓶颈层，可以有效地降低计算量。

![The problem of computational cost.png](img/The problem of computational cost.png)

![Using 1×1 convolution.png](img/Using 1×1 convolution.png)

### 1.7 Inception网络

Inception模块使用到了上面的组件，对于3×3和5×5这样的卷积层，会在之前加入一层1×1的卷积作为瓶颈层，减少计算；而对于池化层，除去同一补全之外，还会在之后增加一层1×1的卷积层，用来缩减通道数量。

![Inception module.png](img/Inception module.png)

Inception网络是由多个Inception模块组合而成的。
![Inception network.png](img/Inception network.png)

一个小彩蛋，Inception的概念，其实就来自盗梦空间。
![WE NEED TO GO DEEPER.png](img/WE NEED TO GO DEEPER.png)

## 2. 在实际项目中使用卷积网络的一些建议

### 2.1 使用开源实现

上面介绍的神经网络架构都比较复杂，在实现的过程中，有很多需要注意的小技巧。对于计算机视觉应用，想要使用上面或其它研究文献中介绍的神经网络架构，通常的建议是去（比如Github上）寻找开源实现，在此基础上进行开发。